In [ ]:
%cd /home/

In [10]:
%pwd

'c:\\Users\\mmadishetty\\Downloads\\Course\\Semester 3\\MLOps\\Assignment\\Assignment\\01_data_pipeline'

In [3]:
%cd ..
%pwd

c:\Users\mmadishetty\Downloads\Course\Semester 3\MLOps\Assignment\Assignment\01_data_pipeline


'c:\\Users\\mmadishetty\\Downloads\\Course\\Semester 3\\MLOps\\Assignment\\Assignment\\01_data_pipeline'

In [1]:

import utils
import constants
import city_tier_mapping
import significant_categorical_level

In [2]:
DB_FILE_NAME = constants.DB_FILE_NAME
DB_PATH = constants.DB_PATH
DATA_DIRECTORY = constants.DATA_DIRECTORY
city_tier_mapping = city_tier_mapping.city_tier_mapping
list_platform = significant_categorical_level.list_platform
list_medium = significant_categorical_level.list_medium
list_source = significant_categorical_level.list_source

INTERACTION_MAPPING = constants.INTERACTION_MAPPING
INDEX_COLUMNS_TRAINING = constants.INDEX_COLUMNS_TRAINING
INDEX_COLUMNS_INFERENCE = constants.INDEX_COLUMNS_INFERENCE
NOT_FEATURES = constants.NOT_FEATURES
utils.build_dbs(DB_FILE_NAME, DB_PATH)
print("running ---- load_data_into_db -----")
utils.load_data_into_db(DB_FILE_NAME, DB_PATH, DATA_DIRECTORY)
print("running ---- map_city_tier -----")
utils.map_city_tier(DB_FILE_NAME, DB_PATH, city_tier_mapping)
print("running ---- map_categorical_vars -----")
utils.map_categorical_vars(DB_FILE_NAME, DB_PATH ,list_platform ,list_medium ,list_source)
print("running ---- interactions_mapping -----")
utils.interactions_mapping(DB_FILE_NAME, DB_PATH, INTERACTION_MAPPING, INDEX_COLUMNS_TRAINING, INDEX_COLUMNS_INFERENCE, NOT_FEATURES)

DB Already Exists
running ---- load_data_into_db -----


C:\Users\mmadishetty\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


running ---- map_city_tier -----
running ---- map_categorical_vars -----
running ---- interactions_mapping -----


In [10]:
import pandas as pd
import os
import logging
import sqlite3
from sqlite3 import Error
import constants
from os.path import isfile
import city_tier_mapping
import significant_categorical_level

DB_FILE_NAME = constants.DB_FILE_NAME
DB_PATH = constants.DB_PATH
filename = os.path.join(DB_PATH, DB_FILE_NAME)

INTERACTION_MAPPING = constants.INTERACTION_MAPPING
INDEX_COLUMNS_TRAINING = constants.INDEX_COLUMNS_TRAINING
INDEX_COLUMNS_INFERENCE = constants.INDEX_COLUMNS_INFERENCE
NOT_FEATURES = constants.NOT_FEATURES
interaction_mapping_pd = pd.read_csv(INTERACTION_MAPPING, index_col=[0])
index_cols_train_dynamic = INDEX_COLUMNS_TRAINING

conn = sqlite3.connect(filename)
df = pd.read_sql_query('SELECT * FROM categorical_variables_mapped', conn)
if 'app_complete_flag' not in df.columns:
    df = df.drop('app_complete_flag', axis=1)
df = pd.melt(df, id_vars=INDEX_COLUMNS_TRAINING, var_name='interaction_type', value_name='interaction_value')#pd.melt(df, id_vars=INDEX_COLUMNS_TRAINING, value_vars=interaction_mapping_pd.keys())
df['interaction_value'] = df['interaction_value'].fillna(0)
df = pd.merge(df, interaction_mapping_pd, on='interaction_type', how='left')
df.drop(['interaction_type'], axis=1)
df_pivot = df.pivot_table(
        values='interaction_value', index=INDEX_COLUMNS_TRAINING, columns='interaction_mapping', aggfunc='sum')#df.pivot_table(index=INDEX_COLUMNS_TRAINING, columns='variable', values='value', aggfunc='first')
df = df.reset_index()
df = df.drop(NOT_FEATURES, axis=1)
df.to_sql('interactions_mapped', conn, if_exists='replace', index=False)
df.to_sql('model_input', conn, if_exists='replace', index=False)

In [ ]:

except Error as e:
    print(e)
finally:
    if conn:
        conn.close()